
## **Carregando os dados**`

In [ ]:
# Importando bibliotecas necessárias
import numpy as np
import pandas as pd
from google.colab import files
from scipy.io import arff
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Fazer upload do arquivo ARFF
uploaded = files.upload()

# O nome do arquivo é a chave no dicionário uploaded
filename = next(iter(uploaded))

# Carregar o arquivo ARFF
data, meta = arff.loadarff(filename)

# Converter para DataFrame do pandas
df = pd.DataFrame(data)

# Exibir os primeiros registros do DataFrame
print(df.head())


# Verificar se há colunas com valores em bytes e convertê-las para string
for col in df.columns:
    if df[col].dtype == 'object':  # Verificando colunas que contêm bytes
        df[col] = df[col].apply(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x)

# Identificar as colunas de atributos (X) e a coluna de classe (y)
X = df.iloc[:, :-1]  # Todas as colunas exceto a última (atributos)
y = df.iloc[:, -1]  # Última coluna (classe)

# Verificar os tipos de dados para identificar colunas numéricas
numerical_cols = X.select_dtypes(include=['float64', 'int64']).columns.tolist()

# Exibir os dados numéricos antes da transformação
print("Dados numéricos antes da transformação:")
print(X[numerical_cols].head())

# Transformar colunas numéricas em categorias utilizando binning
for col in numerical_cols:
    # Aplicar pd.cut para discretização em 4 categorias (bins)
    X[col] = pd.cut(X[col], bins=4, labels=['Muito Baixo', 'Baixo', 'Alto', 'Muito Alto'])

# Exibir os dados após a transformação
print("\nDados após a transformação para categóricos:")
print(X.head())

# Aplicar Label Encoding nas novas colunas categóricas
label_encoder = LabelEncoder()
for col in X.columns:
    X[col] = label_encoder.fit_transform(X[col])

# Exibir os dados após Label Encoding
print("\nDados após Label Encoding:")
print(X.head())



Saving credit-g.arff to credit-g.arff
  checking_status  duration                     credit_history  \
0           b'<0'       6.0  b'critical/other existing credit'   
1     b'0<=X<200'      48.0                   b'existing paid'   
2  b'no checking'      12.0  b'critical/other existing credit'   
3           b'<0'      42.0                   b'existing paid'   
4           b'<0'      24.0              b'delayed previously'   

                  purpose  credit_amount       savings_status employment  \
0             b'radio/tv'         1169.0  b'no known savings'     b'>=7'   
1             b'radio/tv'         5951.0              b'<100'  b'1<=X<4'   
2            b'education'         2096.0              b'<100'  b'4<=X<7'   
3  b'furniture/equipment'         7882.0              b'<100'  b'4<=X<7'   
4              b'new car'         4870.0              b'<100'  b'1<=X<4'   

   installment_commitment        personal_status other_parties  ...  \
0                     4.0         b'm

## **Naive Bayes com holdout sem repetição**

In [ ]:
# Dividir os dados em treino (80%) e teste (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=None)  # Ajuste random_state se necessário

# Definir o modelo Naive Bayes
naive_bayes = MultinomialNB()

# Treinar o modelo
naive_bayes.fit(X_train, y_train)

# Fazer previsões
y_pred = naive_bayes.predict(X_test)

# Avaliar a acurácia do modelo
accuracy = accuracy_score(y_test, y_pred)

# Exibir os resultados
print(f"Acurácia do modelo Naive Bayes: {accuracy * 100:.2f}%")

Acurácia do modelo Naive Bayes: 71.00%


## **Naive Bayes com holdout com repetições**

In [ ]:
# Definir o modelo Naive Bayes
naive_bayes = MultinomialNB()


# Inicializar listas para armazenar as acurácias
accuracies = []

# Realizar Holdout com 30 repetições
for _ in range(30):
    # Dividir os dados em treino (80%) e teste (20%)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=None)  # random_state=None para diferentes divisões

    # Treinar o modelo
    naive_bayes.fit(X_train, y_train)

    # Fazer previsões
    y_pred = naive_bayes.predict(X_test)

    # Avaliar a acurácia do modelo
    accuracy = accuracy_score(y_test, y_pred)
    accuracies.append(accuracy)

# Calcular a média e o desvio padrão das acurácias
mean_accuracy = np.mean(accuracies)
std_accuracy = np.std(accuracies)

# Exibir os resultados
print(f"Acurácia média do Naive Bayes (Holdout): {mean_accuracy * 100:.2f}%")
print(f"Desvio padrão da acurácia: {std_accuracy * 100:.2f}%")

Acurácia média do Naive Bayes (Holdout): 69.88%
Desvio padrão da acurácia: 3.76%


## **Naive Bayes com validação cruzada**

In [ ]:
# Definir o modelo Naive Bayes
naive_bayes = MultinomialNB()

# Realizar validação cruzada com 10 grupos
cv_scores = cross_val_score(naive_bayes, X, y, cv=10)

# Exibir a acurácia média e desvio padrão da validação cruzada
print(f"Acurácia média da validação cruzada: {cv_scores.mean() * 100:.2f}%")
print(f"Desvio padrão da acurácia: {cv_scores.std() * 100:.2f}%")

Acurácia média da validação cruzada: 69.50%
Desvio padrão da acurácia: 2.80%
